In [ ]:
print(checkpoint)

In [1]:
print(logging)

NameError: name 'logging' is not defined

In [2]:
model

NameError: name 'model' is not defined

In [3]:
log_dir

NameError: name 'log_dir' is not defined

In [4]:
model = create_model(input_shape, anchors, num_classes,load_pretrained=False,
                            weights_path=MY_PATH+'logs/ep009-loss6.645-val_loss7.111.h5',
            freeze_body=2)

NameError: name 'create_model' is not defined

In [5]:
from keras.models import load_model
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import os
from yolo3_MobileNet.mobilenet import preprocess_true_boxes, yolo_body, yolo_loss
from yolo3_MobileNet.utils import get_random_data

In [6]:
MY_PATH = '/Users/akbya/Downloads/'

    train_path = MY_PATH+ '/Ornithomate/train_data.txt'
    val_path = MY_PATH+ 'Ornithomate/val_data.txt'
    log_dir = MY_PATH+ 'Ornithomate/logs/'
    classes_path = MY_PATH+ 'Ornithomate/classes.txt'
    anchors_path = MY_PATH+ 'Ornithomate-main/yolo_anchors.txt'
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)

    input_shape = (320,320) # multiple of 32, hw

    model = create_model(input_shape, anchors, num_classes,load_pretrained=False,
                            weights_path=MY_PATH+'logs/ep009-loss6.645-val_loss7.111.h5',
            freeze_body=2)

    model.summary()

IndentationError: unexpected indent (<ipython-input-6-90b8b075d63b>, line 3)

In [7]:
MY_PATH = '/Users/akbya/Downloads/'

train_path = MY_PATH+ '/Ornithomate/train_data.txt'
val_path = MY_PATH+ 'Ornithomate/val_data.txt'
log_dir = MY_PATH+ 'Ornithomate/logs/'
classes_path = MY_PATH+ 'Ornithomate/classes.txt'
anchors_path = MY_PATH+ 'Ornithomate-main/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (320,320) # multiple of 32, hw

model = create_model(input_shape, anchors, num_classes,load_pretrained=False,
                        weights_path=MY_PATH+'logs/ep009-loss6.645-val_loss7.111.h5',
        freeze_body=2)

model.summary()

NameError: name 'get_classes' is not defined

In [8]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def create_model(input_shape, anchors, num_classes, load_pretrained=False, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    print(model_body.output)
    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [9]:

MY_PATH = '/Users/akbya/Downloads/'

train_path = MY_PATH+ '/Ornithomate/train_data.txt'
val_path = MY_PATH+ 'Ornithomate/val_data.txt'
log_dir = MY_PATH+ 'Ornithomate/logs/'
classes_path = MY_PATH+ 'Ornithomate/classes.txt'
anchors_path = MY_PATH+ 'Ornithomate-main/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (320,320) # multiple of 32, hw

model = create_model(input_shape, anchors, num_classes,load_pretrained=False,
                        weights_path=MY_PATH+'logs/ep009-loss6.645-val_loss7.111.h5',
        freeze_body=2)

model.summary()

Create YOLOv3 model with 9 anchors and 10 classes.
[<KerasTensor: shape=(None, None, None, 45) dtype=float32 (created by layer 'conv2d_6')>, <KerasTensor: shape=(None, None, None, 45) dtype=float32 (created by layer 'conv2d_14')>, <KerasTensor: shape=(None, None, None, 45) dtype=float32 (created by layer 'conv2d_22')>]
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                 

In [11]:
logging = TensorBoard(log_dir=log_dir)
# checkpoint = ModelCheckpoint(log_dir + 'car_mobilenet_yolov3.ckpt',
#    monitor='val_loss', save_weights_only=False, period=1)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

with open(train_path) as t_f:
    t_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(t_lines)
np.random.seed(None)
#train_test_pourcentage = int(len(t_lines)*0.8)
#v_lines = t_lines[train_test_pourcentage:]
#t_lines = t_lines[:train_test_pourcentage]
num_train = len(t_lines)
with open(val_path) as v_f:
        v_lines = v_f.readlines()
np.random.seed(10010)
np.random.shuffle(v_lines)
np.random.seed(None)
num_val = len(v_lines)

In [13]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

batch_size = 16

In [17]:
model = create_model(input_shape, anchors, num_classes,load_pretrained=True,
                            weights_path=log_dir+'ep009-loss6.645-val_loss7.111.h5',
            freeze_body=2)

Create YOLOv3 model with 9 anchors and 10 classes.
Load weights /Users/akbya/Downloads/Ornithomate/logs/ep009-loss6.645-val_loss7.111.h5.
Freeze the first 150 layers of total 153 layers.
[<KerasTensor: shape=(None, None, None, 45) dtype=float32 (created by layer 'conv2d_6')>, <KerasTensor: shape=(None, None, None, 45) dtype=float32 (created by layer 'conv2d_14')>, <KerasTensor: shape=(None, None, None, 45) dtype=float32 (created by layer 'conv2d_22')>]


In [19]:

model.compile(optimizer=Adam(learning_rate=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

batch_size = 16

In [20]:
model.fit(data_generator_wrapper(t_lines, batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(v_lines, batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=1,
            initial_epoch=0,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])

99/99 [==============================] - 157s 2s/step - loss: 6.6890 - val_loss: 7.0465 - lr: 0.0010


In [28]:
train_path = MY_PATH+ '/Ornithomate/test_data.txt'
with open(train_path) as t_f:
    test_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(test_lines)
np.random.seed(None)
num_test = len(test_lines)
if True:
    print('Test on {} samples with batch size {}.'.format(num_test, batch_size))
    model.evaluate(test_lines, batch_size)

Test on 199 samples with batch size 16.


IndexError: list index out of range

In [30]:
test_path = MY_PATH+ '/Ornithomate/test_data.txt'
with open(test_path) as t_f:
    test_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(test_lines)
np.random.seed(None)
num_test = len(test_lines)
results = model.evaluate(test_lines, batch_size=10)
print("test loss", results)

ValueError: in user code:

    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1471, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\akbya\

In [31]:
test_path = MY_PATH+ '/Ornithomate/test_data.txt'
with open(test_path) as t_f:
    test_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(test_lines)
np.random.seed(None)
num_test = len(test_lines)
results = model.predict(test_lines[:,1], batch_size=10)
print("test loss", results)

TypeError: list indices must be integers or slices, not tuple

In [32]:
test_path = MY_PATH+ '/Ornithomate/test_data.txt'
with open(test_path) as t_f:
    test_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(test_lines)
np.random.seed(None)
num_test = len(test_lines)
results = model.predict(test_lines[:,1])
print("test loss", results)

TypeError: list indices must be integers or slices, not tuple

In [35]:
test_path = MY_PATH+ '/Ornithomate/test_data.txt'
with open(test_path) as t_f:
    test_lines = t_f.readlines()
np.random.seed(10101)
np.random.shuffle(test_lines)
np.random.seed(None)
num_test = len(test_lines)
results = model.predict(test_lines)
print("test loss", results)

ValueError: in user code:

    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\akbya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users